## 주의
- 파일 경로 설정부탁드려요


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# # !rm ~/.cache/matplotlib -rf

### 라이브러리 불러오기

In [3]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,RobustScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
plt.rc('font', family='NanumBarunGothic') 

plt.style.use('fivethirtyeight')

### 데이터 불러오기

In [4]:
cus = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/cus_info.csv")
iem = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/iem_info_20210902.csv")
hist = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/stk_bnc_hist.csv")
train = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/stk_hld_train.csv")
test = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/stk_hld_test.csv")

submission = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/sample_submission.csv")

In [5]:
usd = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/원달러환율.csv')
kospi = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/코스피.csv')
kosd = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/코스닥.csv')
sp = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/s&p500.csv')
ns = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/나스닥지수.csv')

In [6]:
cpi = pd.read_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/소비자심리지수.csv')
var = pd.read_csv("/content/drive/MyDrive/DAYCON/NH투자증권/DATA/train_var.csv")

In [7]:
kospi.drop(['거래량'],axis=1,inplace=True)
kosd.drop(['거래량'],axis=1,inplace=True)
sp.drop(['거래량'],axis=1,inplace=True)
ns.drop(['거래량'],axis=1,inplace=True)
var.drop('Unnamed: 0',axis=1, inplace = True)
var.rename(columns = {'날짜' : '매수일자'}, inplace=True)

### 데이터 명세 한글 전환

In [8]:
cus = cus.rename(columns = {'act_id':'계좌ID', 'sex_dit_cd':'성별', 'cus_age_stn_cd':'연령대', 'ivs_icn_cd':'투자성향',
       'cus_aet_stn_cd':'자산구간', 'mrz_pdt_tp_sgm_cd':'주거래상품군', 'lsg_sgm_cd':'Life Stage', 'tco_cus_grd_cd':'서비스등급',
       'tot_ivs_te_sgm_cd':'총 투자기간', 'mrz_btp_dit_cd':'주거래업종구분'})

iem = iem.rename(columns = {'iem_cd':'종목코드', 'iem_krl_nm':'종목한글명', 'btp_cfc_cd':'종목업종', 'mkt_pr_tal_scl_tp_cd':'시가총액 규모유형',
       'stk_dit_cd':'시장구분'})

hist = hist.rename(columns = {'act_id' : '계좌ID', 'bse_dt':'기준일자', 'iem_cd':'종목코드', 'bnc_qty':'잔고수량', 'tot_aet_amt':'잔고금액', 'stk_par_pr':'액면가'})

train = train.rename(columns = {'act_id':'계좌ID', 'iem_cd':'종목코드', 'byn_dt':'매수일자', 'hold_d':'보유기간'})
test = test.rename(columns = {'act_id':'계좌ID', 'iem_cd':'종목코드', 'byn_dt':'매수일자', 'hist_d':'과거 보유일', 'submit_id':'제출ID', 'hold_d':'보유기간' })

In [9]:
train["hist_d"] = train["보유기간"] * 0.6
train.hist_d = np.trunc(train["hist_d"])

In [10]:
train_data = pd.merge(train, cus, how = "left", on = ["계좌ID"])
train_data = pd.merge(train_data, iem, how = "left", on = ["종목코드"])

test_data = pd.merge(test, cus, how = "left", on = ["계좌ID"])
test_data = pd.merge(test_data, iem, how = "left", on = ["종목코드"])

In [11]:
train_label = train["보유기간"]
#train_data.drop(["보유기간"], axis = 1, inplace = True)

In [12]:
hist["잔고금액 / 잔고수량"] = hist["잔고금액"] / hist["잔고수량"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["계좌ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])
train_data.drop(['종목한글명','기준일자'], axis=1 , inplace=True)
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [13]:
def make_datetime(data, date_column, name=None):
  data[date_column] = data[date_column].astype(str)
  data[date_column] = pd.to_datetime(data[date_column])
  data = data.sort_values(by=date_column, ascending=True)

  if name == 'new':

    data['year'] = data[date_column].dt.year
    data['month'] = data[date_column].dt.month
    data['quater'] = data[date_column].dt.quarter
    data['quater_ver2'] = data['quater'] + (data.year - data.year.min()) * 4


  
  data.reset_index(drop=True, inplace=True)
  
  return data

In [14]:
train = make_datetime(train_data, '매수일자', name='new')
test = make_datetime(test_data, '매수일자', name='new')
usd = make_datetime(usd, '날짜')
kosd= make_datetime(kosd, '날짜')
kospi = make_datetime(kospi,'날짜')
sp = make_datetime(sp, '날짜')
ns = make_datetime(ns, '날짜')
var = make_datetime(var, '매수일자')

In [15]:
x = pd.merge(usd, kosd, how='inner', on='날짜')
x = pd.merge(x, kospi,how='inner', on='날짜')
x = pd.merge(x, sp,how='left', on='날짜')
x = pd.merge(x, ns,how='left', on='날짜')

In [16]:
test = pd.merge(test, x, how='inner', left_on='매수일자', right_on='날짜')
train = pd.merge(train, x, how='inner', left_on='매수일자', right_on='날짜')

In [17]:
train.drop('날짜',axis=1, inplace=True)
test.drop(['날짜','제출ID'], axis= 1, inplace=True)

In [18]:
train.columns = ['계좌ID', '종목코드', '매수일자', '보유기간', 'hist_d', '성별', '연령대', '투자성향', '자산구간',
       '주거래상품군', 'Life Stage', '서비스등급', '총 투자기간', '주거래업종구분', '종목업종',
       '시가총액 규모유형', '시장구분', '잔고수량', '잔고금액', '액면가', '잔고금액 / 잔고수량', 'year',
       'month', 'quater', 'quater_ver2', '원달러환율', '코스닥', '코스피','s&p','나스닥']

test.columns = ['계좌ID', '종목코드', '매수일자', '과거 보유일', '보유기간', '성별', '연령대', '투자성향', '자산구간',
       '주거래상품군', 'Life Stage', '서비스등급', '총 투자기간', '주거래업종구분', '종목한글명', '종목업종',
       '시가총액 규모유형', '시장구분', '기준일자', '잔고수량', '잔고금액', '액면가', '잔고금액 / 잔고수량',
       'year', 'month', 'quater', 'quater_ver2', '원달러환율', '코스닥', '코스피','s&p','나스닥']

In [19]:
test = pd.merge(test, cpi, on=['year','month'])
train = pd.merge(train, cpi, on=['year','month'])

test = pd.merge(test, iem, how='inner', on = ['종목코드', '종목업종', '시가총액 규모유형','시장구분'])
test = pd.merge(test, var, how='left', on=['매수일자', '시장구분', '시가총액 규모유형'])
train = pd.merge(train, var, how='left', on=['매수일자', '시장구분', '시가총액 규모유형'])
test.drop(['quater','종목한글명_y'], axis=1, inplace=True)
train.drop('quater', axis=1, inplace=True)

In [20]:
train = train.sort_values(['계좌ID', '종목코드']).reset_index().drop('index',axis=1)
test = test.sort_values(['계좌ID', '종목코드']).reset_index().drop('index',axis=1)

In [21]:
train['존버'] = 0
train['잔고금액 변화'] = 0
train['잔고수량 차'] = 0

for i in range(len(train) - 1):
  if (train['종목코드'][i]) == (train['종목코드'][i + 1]):
    train['존버'][i + 1] = train['매수일자'][i + 1] - train['매수일자'][i]
    train['잔고금액 변화'][i + 1] = train['잔고금액'][i + 1] - train['잔고금액'][i]
    train['잔고수량 차'][i + 1] = train['잔고수량'][i + 1] - train['잔고수량'][i]

train['존버'] = pd.to_timedelta(train['존버'])
train['존버'] = train['존버'].astype(str)
for num,date in enumerate(train['존버']):
  train['존버'][num] = date.split('days')[0]
train['존버'] = train['존버'].astype(int)

In [29]:
train_copy['존버'] = pd.to_timedelta(train_copy['존버'])
train_copy['존버'] = train_copy['존버'].astype(str)
for num,date in enumerate(train_copy['존버']):
  train_copy['존버'][num] = date.split('days')[0]
train_copy['존버'] = train_copy['존버'].astype(int)

In [35]:
test['존버'] = 0
test['잔고금액 변화'] = 0
test['잔고수량 차'] = 0

for i in range(len(test) - 1):
  if (test['종목코드'][i]) == (test['종목코드'][i + 1]):
    test['존버'][i + 1] = test['매수일자'][i + 1] - test['매수일자'][i]
    test['잔고금액 변화'][i + 1] = test['잔고금액'][i + 1] - test['잔고금액'][i]
    test['잔고수량 차'][i + 1] = test['잔고수량'][i + 1] - test['잔고수량'][i]

test['존버'] = pd.to_timedelta(test['존버'])
test['존버'] = test['존버'].astype(str)
for num,date in enumerate(test['존버']):
  test['존버'][num] = date.split('days')[0]
test['존버'] = test['존버'].astype(int)

In [36]:
train.to_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/final_train.csv', encoding='utf-8')
test.to_csv('/content/drive/MyDrive/DAYCON/NH투자증권/DATA/final_test.csv', encoding='utf-8')